In [2]:
from unityagents import UnityEnvironment
import numpy as np
import random
from collections import namedtuple, deque
import copy
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt

from maddpg import MADDPG


In [3]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [4]:
maddpg = MADDPG(state_size=24, action_size=2, num_agents=2, random_seed=17)

In [5]:
env = UnityEnvironment(file_name="Tennis.app")

# Get the brain name and brain from the environment
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

Mono path[0] = '/Users/cassimiro/code/projects/Multi-Agent-Tennis-Unity/RL-Multi-Agent-Tennis-Unity/Tennis.app/Contents/Resources/Data/Managed'
Mono config path = '/Users/cassimiro/code/projects/Multi-Agent-Tennis-Unity/RL-Multi-Agent-Tennis-Unity/Tennis.app/Contents/MonoBleedingEdge/etc'


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [6]:
def play(n_episodes=10):
    # Load the saved agent weights
    for idx, agent in enumerate(maddpg.agents):
        agent.actor_local.load_state_dict(torch.load(f'agent{idx}_actor.pth'))
        agent.critic_local.load_state_dict(torch.load(f'critic{idx}_critic.pth'))

    # Iterate through the episodes
    for i_episode in range(n_episodes):
        # Reset the environment for each episode
        env_info = env.reset(train_mode=False)[brain_name]
        state = env_info.vector_observations
        scores = np.zeros(2)

        for _ in range(1000):
            # Get actions for the current state from the MADDPG model without adding noise
            action = maddpg.act(state, i_episode, add_noise=False)
            env_info = env.step(action)[brain_name]

            # Obtain the next state, reward, and done flag from the environment
            next_state = env_info.vector_observations
            reward = env_info.rewards
            done = env_info.local_done

            # Update the scores
            scores += reward

            # Terminate the episode if any agent is done
            if np.any(done):
                break

            # Update the current state
            state = next_state

        print(f'Episode {i_episode + 1}, Score: {np.max(scores)}')

# Call the play function to see the performance of the trained agents
play(n_episodes=1)

env.close()


Episode 1, Score: 2.600000038743019
